In [ ]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import Database
import model_definitions
import pandas as pd
import Generator_OneCar as Generator 

### Open input file

In [2]:
SEQUENCE_LENGTH=30
ONE_CAR=True
PATH=r"C:\Users\dansp\OneDrive\Desktop\Tesi\Logs\OneCar_Pos_Vel_Rot.txt"
BATCH_SIZE=256*5
FEATURES=5 

In [3]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [4]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
df=Generator.single_care_dataframe(PATH)

In [5]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

dfs=[]
for i in range(len(df)):
    dfs.append(SubtractDF(df[i]))

united=dfs[0]
for i in range(1,len(df)):
    united=pd.concat([united,dfs[i] ],ignore_index=True)

In [6]:
df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [7]:
df_train.tail()

,DIFF_X,DIFF_Z,DIFF_VEL_X,DIFF_VEL_Z,DIFF_ROT,TIME,RACE,GROUP
7294685,-0.2441,-1.4616,-1.105577,0.44929,2.9616,0.06,448,2
7294686,-0.2908,-1.4482,-0.840877,0.14517,2.3603,0.06,448,2
7294687,-0.3010,-1.4370,0.633449,0.14549,0.2663,0.06,448,2
7294688,-0.3106,-1.4134,-0.336976,0.49492,-0.2922,0.06,448,2
7294689,-0.2937,-1.3714,0.162751,0.70257,-0.6477,0.06,448,2


In [8]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,16000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,16000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,16000)


Length: 7294690 races: 448 n batches: 7240930 / 1280
Length: 3647349 races: 448 n batches: 3593589 / 1280
Length: 3648017 races: 448 n batches: 3594257 / 1280


### Model definition

#### Train and print

In [9]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [ ]:
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=1,verbose=1,name="MLP",path="logs_deltaFixedUpdate_Iloc")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0005,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [10]:
lr =1e-5
units=32
cells=1
normalization=False
name=f"LSTM{cells}_{units}_{lr}_Norm{normalization}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="logs_NoPos",
                                                     epoch=5, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

Epoch 1/5
  54/5655 [..............................] - ETA: 30:01 - loss: 1.9848 - mean_absolute_error: 1.0862 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
tf.saved_model.save(mlp_ph,f'saved_model/MLP_Iloc')

In [ ]:
model=model_definitions.LSTM_Model(0.2, cells=3,units=32, normalization=False)
model.load_weights("saved_model\LSTM3_32_1e-05_NormFalse")

In [ ]:
x=df_test.copy()
x.drop(["TIME","RACE","GROUP"],axis=1,inplace=True)
x_ten=tf.constant(x[:SEQUENCE_LENGTH])
x_ten=tf.reshape(x_ten,(1,SEQUENCE_LENGTH,FEATURES))

In [ ]:
y_p=model.predict(x_ten)

y_p

In [ ]:
y=x.iloc[SEQUENCE_LENGTH+1]
diff= y.values-y_p
print(y)
diff